# Pandas и большие файлы

# Домашнее задание

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv ('ml-latest-small/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
result = ratings.groupby('userId').agg({'timestamp': ['max', 'min'], 'userId':'count'}).reset_index()
result['lifetime'] = ((result['timestamp']['max'] - result['timestamp']['min']) / 3600).round(1) #среднее время жизни пользователя в часах
result.loc[result['userId']['count'] > 100,  ['lifetime','userId']]

userId        lifetime
           count          
3        4   204      56.5
7        8   116      23.7
14      15  1700  130942.6
16      17   363       2.2
18      19   423       1.5
..     ...   ...       ...
655    656   128       0.8
658    659   142    8780.4
663    664   519   27272.3
664    665   434   15036.5
670    671   115    3134.4

[258 rows x 3 columns]

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [9]:
table_1 = rzd.merge(auto, how='outer').merge(air, how='outer')
table_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [10]:
table_2 = client_base.merge(rzd, how='left').merge(auto, how='left').merge(air, how='left')
table_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Ответ: 
Стоит взять  самую раннюю геометку, она скорее всего будет равна положению пользователя в момент первого визита в приложение, а последняя метка будет уже в момент совершения покупки или проверки доставки. Основной нюанс в дубликатах, если человек несколько раз заказывал товар, как именно определить какое местоположение к чему относилось? Может быть тут помогла бы метка времени, чтобы по ней и id соединить данные.
Из координат можно получить адрес, первое, что смогла найти для python библиотека geopy https://geopy.readthedocs.io/en/latest/
По адресу можно понять из какой страный или города чаще всего соверщают покупки и визиты. Далее можно взять эти данные, и категорию покупки, и уже на их основе делать рекламу + по источнику откуда пришел пользователь, делать рекламу для конкретного города на определенный источник. Это поможет грамотрее распределить бюджет на рекламу.